## Libraries required to run the project:

#### numpy == 1.15.4
#### opencv_python == 4.1.1.26

### How to run each task:

### For each task, the path folders must already exist!
### Task 1: 

#### script: TemaComputerVision.ipynb
#### function: task_1(input_folder_name, output_folder_name, auxiliary_items_folder_name), where input_folder_name is the path to the folder containing the images (.jpg) for task_1, output_folder_name is the path to the folder where the output files will be stored and auxiliary_items_folder_name is the path to the folder with the auxiliary resources required by the program ( the "optiuni" folder)

#### output: the output files for each image will be stored in output_folder_name/X_Y.txt where X will be the index of the test and Y will be the index of the photo from the test above mentioned.

### Task 2: 
#### script: TemaComputerVision.ipynb
#### function: task_2(input_folder_name, output_folder_name, auxiliary_items_folder_name), where input_folder_name is the path to the folder containing the images (.jpg) for task_1, output_folder_name is the path to the folder where the output files will be stored and auxiliary_items_folder_name is the path to the folder with the auxiliary resources required by the program ( the "optiuni" folder)

#### output: the output files for each image will be stored in output_folder_name/X_Y.txt where X will be the index of the test and Y will be the index of the photo from the test above mentioned.

### Task 3: 
#### script: TemaComputerVision.ipynb
#### function: task_3(input_folder_name, output_folder_name, auxiliary_items_folder_name), where input_folder_name is the path to the folder containing the images (.jpg) for task_1, output_folder_name is the path to the folder where the output files will be stored and auxiliary_items_folder_name is the path to the folder with the auxiliary resources required by the program ( the "optiuni" folder)

#### output: the output files for each image will be stored in output_folder_name/X_Y.txt where X will be the index of the test and Y will be the index of the photo from the test above mentioned.

### All tasks: 

#### script: TemaComputerVision.ipynb
#### function: all_tasks(input_folder_name, output_folder_name, auxiliary_items_folder_name), where input_folder_name is the path to the folder containing the images (.jpg) for task_1, output_folder_name is the path to the folder where the output files will be stored and auxiliary_items_folder_name is the path to the folder with the auxiliary resources required by the program ( the "optiuni" folder)

#### output: the output files for each image will be stored in output_folder_name/X_Y.txt where X will be the index of the test and Y will be the index of the photo from the test above mentioned.

In [1]:
import cv2 as cv
import numpy as np
import os

In [2]:
def show_image(title,image):
    image=cv.resize(image,(0,0),fx = 0.25,fy = 0.25)
    cv.imshow(title,image)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [3]:
def extrage_careu(image):
    aux_image = image.copy()
    image = cv.cvtColor(image,cv.COLOR_BGR2GRAY)
    image_m_blur = cv.medianBlur(image,3)
    image_g_blur = cv.GaussianBlur(image_m_blur, (0, 0), 5) 
    image_sharpened = cv.addWeighted(image_m_blur, 1.1, image_g_blur, -0.9, 0)
    _, thresh = cv.threshold(image_sharpened, 47, 255, cv.THRESH_BINARY)

    
    kernel = np.ones((1, 10), np.uint8)
    thresh = cv.dilate(thresh, kernel)
    
    kernel = np.ones((5, 1), np.uint8)
    thresh = cv.dilate(thresh, kernel)

    edges =  cv.Canny(thresh , 100, 255)

    contours, _ = cv.findContours(edges,  cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    max_area = 0
    
    #show_image("Margini", edges)
   
    for i in range(len(contours)):
        if(len(contours[i]) >3):
            possible_top_left = None
            possible_bottom_right = None
            for point in contours[i].squeeze():
                if possible_top_left is None or point[0] + point[1] < possible_top_left[0] + possible_top_left[1]:
                    possible_top_left = point

                if possible_bottom_right is None or point[0] + point[1] > possible_bottom_right[0] + possible_bottom_right[1] :
                    possible_bottom_right = point

            diff = np.diff(contours[i].squeeze(), axis = 1)
            possible_top_right = contours[i].squeeze()[np.argmin(diff)]
            possible_bottom_left = contours[i].squeeze()[np.argmax(diff)]
            
            if cv.contourArea(np.array([[possible_top_left],[possible_top_right],[possible_bottom_right],[possible_bottom_left]])) > max_area:
                max_area = cv.contourArea(np.array([[possible_top_left],[possible_top_right],[possible_bottom_right],[possible_bottom_left]]))
                top_left = possible_top_left
                bottom_right = possible_bottom_right
                top_right = possible_top_right
                bottom_left = possible_bottom_left

    width = 2100
    height = 2100
    
    top_left[0] = min(top_left[0], bottom_left[0])
    bottom_left[0] = min(top_left[0], bottom_left[0])
    
    top_right[0] = max(top_right[0], bottom_right[0])
    bottom_right[0] = max(top_right[0], bottom_right[0])
    
    top_left[1] = min(top_left[1], top_right[1])
    top_right[1] = min(top_left[1], top_right[1])
    
    bottom_left[1] = max(bottom_left[1], bottom_right[1])
    bottom_right[1] = max(bottom_left[1], bottom_right[1])
    
    
    image_copy = aux_image.copy()
    cv.circle(image_copy,tuple(top_left),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(top_right),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(bottom_left),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(bottom_right),20,(0,0,255),-1)
    #show_image("detected corners",image_copy)

    puzzle = np.array([top_left, top_right, bottom_right, bottom_left], dtype = "float32")
    destination_of_puzzle = np.array([[0, 0], [width, 0], [width, height], [0, height]], dtype = "float32")
    
    M = cv.getPerspectiveTransform(puzzle, destination_of_puzzle)
    
    result = cv.warpPerspective(aux_image, M, (width, height))
    #result = cv.cvtColor(result, cv.COLOR_GRAY2BGR)
    return result

In [4]:
def get_matrix(image, options):
#     image = cv.cvtColor(image,cv.COLOR_BGR2GRAY)
#     image_m_blur = cv.medianBlur(image,3)
#     image_g_blur = cv.GaussianBlur(image_m_blur, (0, 0), 5) 
#     image_sharpened = cv.addWeighted(image_m_blur, 1.1, image_g_blur, -0.9, 0)
#     _, thresh = cv.threshold(image_sharpened, 47, 255, cv.THRESH_BINARY)
    
    matrix = []
    #show_image("IMM", image)
    for i in range(0, len(image[0]), len(image[0]) // 15):

        lain = []
        for j in range(0, len(image[1]), len(image[1]) // 15):

            crop_img = image[i : i + len(image[0]) // 15, j : j + len(image[1]) // 15]
            crop_img = cv.cvtColor(crop_img,cv.COLOR_BGR2GRAY)
            #show_image("Cropare", crop_img)
            
            #print("Suntem la imaginea", i, j, sep = " ")
            path = options
            files = os.listdir(path)
            maxx = - np.inf
            best = ""
            for file in files:
                aux_img = cv.imread(path + file)
                aux_img = cv.cvtColor(aux_img,cv.COLOR_BGR2GRAY)
                #show_image("Comparare", aux_img)
#                 distance = np.inf
#                 for ii in range(len(crop_img)):
#                     for jj in range(len(crop_img[ii])):
#                         if ii + len(aux_img) > len(crop_img) or jj + len(aux_img[0]) > len(crop_img[ii]):
#                             continue
                        
#                         aux = np.sum((crop_img[ii : ii + len(aux_img), jj : jj + len(aux_img[0])] - aux_img))
#                         distance = min(distance, aux)
                        
                        #print("Distanta la indicii " + str(i) + " " + str(j) + " este " + str(distance))
                            
                #print("Distanta fata de " + file + " este " + str(distance))
#                 if distance < minn:
#                     minn = distance
#                     best = file
                corr = cv.matchTemplate(crop_img, aux_img,  cv.TM_CCOEFF_NORMED)
                corr = np.max(corr)
                #print("Matchul maxim cu " + file + " este " + str(corr))
                if corr > maxx:
                    maxx = corr
                    best = file
            #letter
            aux_imag = cv.imread(path + best)
            #print("Match maximal:", best)
            
            if maxx > 0.75: 
                lain.append(best[0])
            else:
                lain.append("_")
                
                
        matrix.append(lain)
    return matrix
    

In [5]:
#img = cv.imread("./CAVA-2022-TEMA1/imagini_auxiliare/litere_1.jpg")
#show_image("imaginelala", img)
# rez = extrage_careu(img)
#show_image("imagine", rez)
# matrix = get_matrix(rez)
    


In [6]:
score_per_letter = {
    "A" : 1,
    "B" : 9,
    "C" : 1,
    "D" : 2,
    "E" : 1,
    "F" : 8,
    "G" : 9,
    "H" : 10,
    "I" : 1,
    "J" : 10,
    "K" : 0,
    "L" : 1,
    "M" : 4,
    "N" : 1,
    "O" : 1,
    "P" : 2,
    "R" : 1,
    "S" : 1,
    "T" : 1,
    "U" : 1,
    "V" : 8,
    "X" : 10,
    "Z" : 10,
}

red_cells = [(0, 0), (0, 7), (0, 14), (7, 0), (7, 14), (14, 0), (14, 7), (14, 14)]

blue_cells = [(1, 5), (1, 9), (5, 1), (5, 5), (5, 9), (5, 13), (9, 1), (9, 5), (9, 9), (9, 13), (13, 5), (13, 9)]

purple_cells = [(1, 1), (2, 2), (3, 3), (4, 4), (1, 13), (2, 12), (3, 11), (4, 10), 
                (13, 1), (12, 2), (11, 3), (10, 4), (10, 10), (11, 11), (12, 12), (13, 13), (7, 7)]

cyan_cells = [(0, 3), (0, 11), (2, 6), (2, 8), (3, 0), (3, 7), (3, 14), (6, 2), (6, 6), (6, 8), (6, 12), (7, 3), (7, 11)
              , (8, 2), (8, 6), (8, 8), (8, 12), (11, 0), (11, 7), (11, 14), (12, 6), (12, 8), (14, 3) , (14, 11)]

def calculate_score_lin(matrix, added, last_added, first_added = (-1, -1)):
    #print("A intrat pe calc score lin!")
    word_multiplier = 1
    if first_added != (-1, -1):
        rez = 0
        cur = first_added
        while True:
            #print("Litera " + str(matrix[cur[0]][cur[1]]))
            if cur in added:
                if cur in red_cells:
                    word_multiplier *= 3
                    rez += score_per_letter[matrix[cur[0]][cur[1]]]
                    #print("Celula rosie deci scor triplat!")
                elif cur in purple_cells:
                    word_multiplier *= 2
                    rez += score_per_letter[matrix[cur[0]][cur[1]]]
                    #print("Celula mov deci scor dublat!")
                elif cur in blue_cells:
                    rez += score_per_letter[matrix[cur[0]][cur[1]]] * 3
                    #print("Celula albastra deci scor celula triplu!")
                elif cur in cyan_cells:
                    rez += score_per_letter[matrix[cur[0]][cur[1]]] * 2
                    #print("Celula bleu deci scor celula dublat!")
                else:
                    rez += score_per_letter[matrix[cur[0]][cur[1]]]
                    #print("Celula portocalie deci scor simplu!")
                    
            else:
                #print("Celula veche portocalie deci scor simplu!")
                rez += score_per_letter[matrix[cur[0]][cur[1]]]
            if cur == last_added:
                break
            cur = (cur[0], cur[1] + 1)
        return rez * word_multiplier
    
    else:
        rez = 0
        st = last_added
        dr = last_added
        
        while st[1] >= 0 and matrix[st[0]][st[1]] != "_":
            st = (st[0], st[1] - 1)
        st = (st[0], st[1] + 1)
        
        while dr[1] < 15 and matrix[dr[0]][dr[1]] != "_":
            dr = (dr[0], dr[1] + 1)
        
        dr = (dr[0], dr[1] - 1)
        cur = st
        last_added = dr
        while True:
            #print("Litera " + str(matrix[cur[0]][cur[1]]))
            if cur in added:
                if cur in red_cells:
                    word_multiplier *= 3
                    rez += score_per_letter[matrix[cur[0]][cur[1]]]
                    #print("Celula rosie deci scor triplat!")
                elif cur in purple_cells:
                    word_multiplier *= 2
                    rez += score_per_letter[matrix[cur[0]][cur[1]]]
                    #print("Celula mov deci scor dublat!")
                elif cur in blue_cells:
                    rez += score_per_letter[matrix[cur[0]][cur[1]]] * 3
                    #print("Celula albastra deci scor celula triplu!")
                elif cur in cyan_cells:
                    rez += score_per_letter[matrix[cur[0]][cur[1]]] * 2
                    #print("Celula bleu deci scor celula dublat!")
                else:
                    rez += score_per_letter[matrix[cur[0]][cur[1]]]
                    #print("Celula portocalie deci scor simplu!")
                
                    
            else:
                #print("Celula veche portocalie deci scor simplu!")
                rez += score_per_letter[matrix[cur[0]][cur[1]]]
            if cur == last_added:
                break
            cur = (cur[0], cur[1] + 1)
        return rez * word_multiplier
    
def calculate_score_col(matrix, added, last_added, first_added = (-1, -1)):
    #print("A intrat pe calc score col!")
    word_multiplier = 1
    if first_added != (-1, -1):
        rez = 0
        cur = first_added
        while True:
            #print("Litera " + str(matrix[cur[0]][cur[1]]))
            if cur in added:
                if cur in red_cells:
                    word_multiplier *= 3
                    rez += score_per_letter[matrix[cur[0]][cur[1]]]
                    #print("Celula rosie deci scor triplat!")
                elif cur in purple_cells:
                    word_multiplier *= 2
                    rez += score_per_letter[matrix[cur[0]][cur[1]]]
                    #print("Celula mov deci scor dublat!")
                elif cur in blue_cells:
                    rez += score_per_letter[matrix[cur[0]][cur[1]]] * 3
                    #print("Celula albastra deci scor celula triplu!")
                elif cur in cyan_cells:
                    rez += score_per_letter[matrix[cur[0]][cur[1]]] * 2
                    #print("Celula bleu deci scor celula dublat!")
                else:
                    rez += score_per_letter[matrix[cur[0]][cur[1]]]
                    #print("Celula portocalie deci scor simplu!")
                    
            else:
                #print("Celula veche portocalie deci scor simplu!")
                rez += score_per_letter[matrix[cur[0]][cur[1]]]
            if cur == last_added:
                break
            cur = (cur[0] + 1, cur[1])
        return rez * word_multiplier
    
    else:
        rez = 0
        st = last_added
        dr = last_added
        
        while st[0] >= 0 and matrix[st[0]][st[1]] != "_":
            st = (st[0] - 1, st[1])
        st = (st[0] + 1, st[1])
        
        while dr[0] < 15 and matrix[dr[0]][dr[1]] != "_":
            dr = (dr[0] + 1, dr[1])
            
        dr = (dr[0] - 1, dr[1])
        
        cur = st
        last_added = dr
        #print("Looping from " + str(cur[0]) + " " + str(cur[1]) + " to " + str(last_added[0]) + " " + str(last_added[1]))
        while True:
            #print("Litera " + str(matrix[cur[0]][cur[1]]))
            if cur in added:
                if cur in red_cells:
                    word_multiplier *= 3
                    rez += score_per_letter[matrix[cur[0]][cur[1]]]
                    #print("Celula rosie deci scor triplat!")
                elif cur in purple_cells:
                    word_multiplier *= 2
                    rez += score_per_letter[matrix[cur[0]][cur[1]]]
                    #print("Celula mov deci scor dublat!")
                elif cur in blue_cells:
                    rez += score_per_letter[matrix[cur[0]][cur[1]]] * 3
                    #print("Celula albastra deci scor celula triplu!")
                elif cur in cyan_cells:
                    rez += score_per_letter[matrix[cur[0]][cur[1]]] * 2
                    #print("Celula bleu deci scor celula dublat!")
                else:
                    rez += score_per_letter[matrix[cur[0]][cur[1]]]
                    #print("Celula portocalie deci scor simplu!")
                    
            else:
                #print("Celula veche portocalie deci scor simplu!")
                rez += score_per_letter[matrix[cur[0]][cur[1]]]
            if cur == last_added:
                break
            cur = (cur[0] + 1, cur[1])
        return rez * word_multiplier

def calculate_score(prev_matrix, matrix):
    added = {}
    on_lin = False
    on_col = False
    last_added = (0, 0)
    first_added = (0, 0)
    score = 0
    flag = True
    for i in range(len(matrix[0])):
            for j in range(len(matrix[i])):
                if matrix[i][j] != prev_matrix[i][j]:
                    
                    if flag:
                        first_added = (i, j)
                        flag = False
                    added[(i, j)] = True
                        
                    last_added = (i, j)
                        
    if len(added) == 7:
        score += 50
    if first_added[0] == last_added[0]:
        on_lin = True
    else:
        on_col = True
        
    if on_lin:
        #print("Suntem pe online!")
        while first_added[1] >= 0 and matrix[first_added[0]][first_added[1]] != '_':
            first_added = (first_added[0], first_added[1] - 1)
        first_added = (first_added[0], first_added[1] + 1)
            
        while last_added[1] < 15 and matrix[last_added[0]][last_added[1]] != '_':
            last_added = (last_added[0], last_added[1] + 1)
        
        last_added = (last_added[0], last_added[1] - 1)
        
        #print("Prima pozitie este " + str(first_added) + " iar ultima este " + str(last_added))
        
        if first_added != last_added:
            score += calculate_score_lin(matrix, added, last_added, first_added)
        cur = last_added
        while cur[0] >= 0 and cur[1] >= 0:
            if cur in added and ((cur[0] > 0 and matrix[cur[0] - 1][cur[1]] != "_") or (cur[0] < 14 and matrix[cur[0] + 1][cur[1]] != "_")):
                score += calculate_score_col(matrix, added, cur)
            if cur == first_added:
                break
            cur = (cur[0], cur[1] - 1)
    else:
        #print("Suntem pe on column!")
        while first_added[0] >= 0 and matrix[first_added[0]][first_added[1]] != '_':
            first_added = (first_added[0] - 1, first_added[1])
        first_added = (first_added[0] + 1, first_added[1])
            
        while last_added[0] < 15 and matrix[last_added[0]][last_added[1]] != '_':
            last_added = (last_added[0] + 1, last_added[1])
        
        last_added = (last_added[0] - 1, last_added[1])
        
        #print("Prima pozitie este " + str(first_added) + " iar ultima este " + str(last_added))
        
        if first_added != last_added:
            score += calculate_score_col(matrix, added, last_added, first_added)
        cur = last_added
        while cur[0] >= 0 and cur[1] >= 0:
            if cur in added and ((cur[1] > 0 and matrix[cur[0]][cur[1] - 1] != "_") or (cur[1] < 14 and matrix[cur[0]][cur[1] + 1] != "_")):
                score += calculate_score_lin(matrix, added, cur)
            if cur == first_added:
                break
            cur = (cur[0] - 1, cur[1])
            
    #print("Scor returnat: ", score)
    return score
                    
    

In [7]:
def task_1(source, dest, options):
    letters = {
    "0" : "A",
    "1" : "B",
    "2" : "C",
    "3" : "D",
    "4" : "E",
    "5" : "F",
    "6" : "G",
    "7" : "H",
    "8" : "I",
    "9" : "J",
    "10" : "K",
    "11" : "L",
    "12" : "M",
    "13" : "N",
    "14" : "O",
    }
    
    files = os.listdir(source)
    ct = 0
    last_matrix = []
    matrix = []
    for file in files:
        if file[-6:] == "01.jpg":
            matrix = [["_" for i in range(15)] for j in range(15)]
        if file[-4:] != ".jpg":
            continue
            
        print("Suntem la imaginea " + file +"!!\n")
        
        img = cv.imread(source + file)
        ct += 1 
        rez = extrage_careu(img)
        last_matrix = matrix
        
        matrix = get_matrix(rez, options)
#         for i in range(len(matrix[0])):
#             for j in range(len(matrix[i])):
#                 print(matrix[i][j], end = " ")
#             print("\n", end = "")
#         print("\n", end = "\n")
        write_file = file[:-4] + ".txt"
        g = open(dest + write_file, "w")
        for i in range(len(matrix[0])):
            for j in range(len(matrix[i])):
                if matrix[i][j] != last_matrix[i][j]:
                    g.write(str(i + 1) + letters[str(j)] + "\n")
   
        g.close()

In [8]:
def task_2(source, dest, options):
    letters = {
    "0" : "A",
    "1" : "B",
    "2" : "C",
    "3" : "D",
    "4" : "E",
    "5" : "F",
    "6" : "G",
    "7" : "H",
    "8" : "I",
    "9" : "J",
    "10" : "K",
    "11" : "L",
    "12" : "M",
    "13" : "N",
    "14" : "O",
    }
    files = os.listdir(source)
    ct = 0
    last_matrix = []
    matrix = []
    for file in files:
        if file[-6:] == "01.jpg":
            matrix = [["_" for i in range(15)] for j in range(15)]
        if file[-4:] != ".jpg":
            continue
        print("Suntem la imaginea " + file +"!!\n")
        img = cv.imread(source + file)
        ct += 1 
        rez = extrage_careu(img)
        last_matrix = matrix
        
        matrix = get_matrix(rez, options)
#         for i in range(len(matrix[0])):
#             for j in range(len(matrix[i])):
#                 print(matrix[i][j], end = " ")
#             print("\n", end = "")
#         print("\n", end = "\n")
        write_file = file[:-4] + ".txt"
        g = open(dest + write_file, "w")
        for i in range(len(matrix[0])):
            for j in range(len(matrix[i])):
                if matrix[i][j] != last_matrix[i][j]:
                    if matrix[i][j] != "K":
                        g.write(str(i + 1) + letters[str(j)] + " " + matrix[i][j] + "\n")
                    else:
                        g.write(str(i + 1) + letters[str(j)] + " ?" + "\n")
                    
        g.close()

In [9]:
def task_3(source, dest, options):
    letters = {
    "0" : "A",
    "1" : "B",
    "2" : "C",
    "3" : "D",
    "4" : "E",
    "5" : "F",
    "6" : "G",
    "7" : "H",
    "8" : "I",
    "9" : "J",
    "10" : "K",
    "11" : "L",
    "12" : "M",
    "13" : "N",
    "14" : "O",
    }
    files = os.listdir(source)
    ct = 0
    last_matrix = []
    matrix = []
    for file in files:
        if file[-6:] == "01.jpg":
            matrix = [["_" for i in range(15)] for j in range(15)]
        if file[-4:] != ".jpg":
            continue
        print("Suntem la imaginea " + file +"!!\n")
        img = cv.imread(source + file)
        ct += 1 
        rez = extrage_careu(img)
        last_matrix = matrix
        
        matrix = get_matrix(rez, options)
#         for i in range(len(matrix[0])):
#             for j in range(len(matrix[i])):
#                 print(matrix[i][j], end = " ")
#             print("\n", end = "")
#         print("\n", end = "\n")
        write_file = file[:-4] + ".txt"
        g = open(dest + write_file, "w")
                    
        g.write(str(calculate_score(last_matrix, matrix)))
                    
        g.close()

In [10]:
def all_tasks(source, dest, options):
    letters = {
    "0" : "A",
    "1" : "B",
    "2" : "C",
    "3" : "D",
    "4" : "E",
    "5" : "F",
    "6" : "G",
    "7" : "H",
    "8" : "I",
    "9" : "J",
    "10" : "K",
    "11" : "L",
    "12" : "M",
    "13" : "N",
    "14" : "O",
    }
    files = os.listdir(source)
    ct = 0
    last_matrix = []
    matrix = []
    for file in files:
        if file[-6:] == "01.jpg":
            matrix = [["_" for i in range(15)] for j in range(15)]
        if file[-4:] != ".jpg":
            continue
        print("Suntem la imaginea " + file +"!!\n")
        img = cv.imread(source + file)
        ct += 1 
        rez = extrage_careu(img)
        last_matrix = matrix
        
        matrix = get_matrix(rez, options)
#         for i in range(len(matrix[0])):
#             for j in range(len(matrix[i])):
#                 print(matrix[i][j], end = " ")
#             print("\n", end = "")
#         print("\n", end = "\n")
        write_file = file[:-4] + ".txt"
        g = open(dest + write_file, "w")
        for i in range(len(matrix[0])):
            for j in range(len(matrix[i])):
                if matrix[i][j] != last_matrix[i][j]:
                    if matrix[i][j] != "K":
                        g.write(str(i + 1) + letters[str(j)] + " " + matrix[i][j] + "\n")
                    else:
                        g.write(str(i + 1) + letters[str(j)] + " ?" + "\n")
                    
        g.write(str(calculate_score(last_matrix, matrix)))
                    
        g.close()

In [11]:
# Testerul meu de acuratete
# def test_solution(source_pred, source_gold):
#     ct = 0
#     for i in range(1, 6):
#         for j in range(1, 21):
#             if j < 10:
#                 f = open(source_pred + str(i) + "_0" + str(j) + ".txt", "r")
#             else:
#                 f = open(source_pred + str(i) + "_" + str(j) + ".txt", "r")
#             if j < 10:
#                 g = open(source_gold + str(i) + "_0" + str(j) + ".txt", "r")
#             else:
#                 g = open(source_gold + str(i) + "_" + str(j) + ".txt", "r")
#             ok = True
#             lines = f.readlines()
#             for ii in range(len(lines)):
#                 line = lines[ii]
#                 aux = g.readline()
#                 #print(aux)
#                 #print(line)
                
#                 if ii != len(lines) - 1:
#                     #print("Intrat la "+ str(i) + " " + str(j))
#                     aux = aux[:-1]
#                     line = line[:-1]
#                 if aux.strip() != line.strip():
#                     print(aux)
#                     print(line)
#                     print("Prost la "+ str(i) + " " + str(j))
#                     ok = False
#                     break
#             if ok:
#                 ct += 1
#             g.close()
#             f.close()
#     return ct / 100

# test_solution("./CAVA-2022-TEMA1/evaluare/fisiere_solutie/352_Buta_Gabriel-Sebastian/", "./CAVA-2022-TEMA1/antrenare/")

In [12]:
# Every task can be rulled through one of these functions, decomment the desired one!

#task_1("./CAVA-2022-TEMA1/antrenare/", "./CAVA-2022-TEMA1/evaluare/fisiere_solutie/352_Buta_Gabriel-Sebastian/", "./CAVA-2022-TEMA1/optiuni/")
#task_2("./CAVA-2022-TEMA1/antrenare/", "./CAVA-2022-TEMA1/evaluare/fisiere_solutie/352_Buta_Gabriel-Sebastian/", "./CAVA-2022-TEMA1/optiuni/")
#task_3("./CAVA-2022-TEMA1/antrenare/", "./CAVA-2022-TEMA1/evaluare/fisiere_solutie/352_Buta_Gabriel-Sebastian/", "./CAVA-2022-TEMA1/optiuni/")
all_tasks("E:/ComputerVision/CAVA-2022-TEMA1/antrenare/", "E:/ComputerVision/CAVA-2022-TEMA1/evaluare/fisiere_solutie/352_Buta_Gabriel-Sebastian/", "E:/ComputerVision/CAVA-2022-TEMA1/optiuni/")

Suntem la imaginea 1_01.jpg!!

Suntem la imaginea 1_02.jpg!!

Suntem la imaginea 1_03.jpg!!

Suntem la imaginea 1_04.jpg!!

Suntem la imaginea 1_05.jpg!!

Suntem la imaginea 1_06.jpg!!

Suntem la imaginea 1_07.jpg!!

Suntem la imaginea 1_08.jpg!!

Suntem la imaginea 1_09.jpg!!

Suntem la imaginea 1_10.jpg!!

Suntem la imaginea 1_11.jpg!!

Suntem la imaginea 1_12.jpg!!

Suntem la imaginea 1_13.jpg!!

Suntem la imaginea 1_14.jpg!!

Suntem la imaginea 1_15.jpg!!

Suntem la imaginea 1_16.jpg!!

Suntem la imaginea 1_17.jpg!!

Suntem la imaginea 1_18.jpg!!

Suntem la imaginea 1_19.jpg!!

Suntem la imaginea 1_20.jpg!!

Suntem la imaginea 2_01.jpg!!

Suntem la imaginea 2_02.jpg!!

Suntem la imaginea 2_03.jpg!!

Suntem la imaginea 2_04.jpg!!

Suntem la imaginea 2_05.jpg!!

Suntem la imaginea 2_06.jpg!!

Suntem la imaginea 2_07.jpg!!

Suntem la imaginea 2_08.jpg!!

Suntem la imaginea 2_09.jpg!!

Suntem la imaginea 2_10.jpg!!

Suntem la imaginea 2_11.jpg!!

Suntem la imaginea 2_12.jpg!!

Suntem l